In [41]:
#importing important libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import sigmoid_kernel

In [42]:
df=pd.read_csv('courses.csv')

In [43]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


In [44]:
df.shape

(3678, 12)

In [45]:
import nltk 
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet') 
  
  
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
  
from nltk.corpus import stopwords 
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'} 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\19058\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\19058\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\19058\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\19058\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
len(df) 
  
finaldata = df[["course_title", "subject"]]            
  
finaldata.head(10) 
finaldata["course_title"][10] 

"Forex Trading Secrets of the Pros With Amazon's AWS"

In [47]:
finaldata.shape

(3678, 2)

In [48]:
#takes few min
def preprocess_sentences(text): 
  text = text.lower() 
  temp_sent =[] 
  words = nltk.word_tokenize(text) 
  tags = nltk.pos_tag(words) 
  for i, word in enumerate(words): 
      if tags[i][1] in VERB_CODES:  
          lemmatized = lemmatizer.lemmatize(word, 'v') 
      else: 
          lemmatized = lemmatizer.lemmatize(word) 
      if lemmatized not in stop_words and lemmatized.isalpha(): 
          temp_sent.append(lemmatized) 
          
  finalsent = ' '.join(temp_sent) 
 
  return finalsent 
  
finaldata["course_title_processed"]= finaldata["course_title"].apply(preprocess_sentences) 
finaldata.head() 

<ipython-input-48-c923e1706e8f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldata["course_title_processed"]= finaldata["course_title"].apply(preprocess_sentences)


,course_title,subject,course_title_processed
0,Ultimate Investment Banking Course,Business Finance,ultimate investment banking course
1,Complete GST Course & Certification - Grow You...,Business Finance,complete gst course certification grow ca prac...
2,Financial Modeling for Business Analysts and C...,Business Finance,financial modeling business analyst consultant
3,Beginner to Pro - Financial Analysis in Excel ...,Business Finance,beginner pro financial analysis excel
4,How To Maximize Your Profits Trading Options,Business Finance,maximize profit trading option


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer 
  
# Vectorizing pre-processed combo using TF-IDF 
tfidfvec = TfidfVectorizer() 
tfidf_course = tfidfvec.fit_transform((finaldata["course_title_processed"])) 

In [50]:
tfidfvec

TfidfVectorizer()

In [51]:
tfidf_course

<3678x3051 sparse matrix of type '<class 'numpy.float64'>'
	with 18009 stored elements in Compressed Sparse Row format>

In [52]:
tfidf_course.shape

(3678, 3051)

In [53]:
# Finding cosine similarity between vectors 
from sklearn.metrics.pairwise import sigmoid_kernel 
sig = cosine_similarity(tfidf_course, tfidf_course)

In [54]:
sig[2]

array([0., 0., 1., ..., 0., 0., 0.])

In [55]:
# Reverse mapping of indices and movie titles
indices = pd.Series(finaldata.index, index=finaldata['course_title']).drop_duplicates()

In [56]:
indices

course_title
Ultimate Investment Banking Course                                0
Complete GST Course & Certification - Grow Your CA Practice       1
Financial Modeling for Business Analysts and Consultants          2
Beginner to Pro - Financial Analysis in Excel 2017                3
How To Maximize Your Profits Trading Options                      4
                                                               ... 
Learn jQuery from Scratch - Master of JavaScript library       3673
How To Design A WordPress Website With No Coding At All        3674
Learn and Build using Polymer                                  3675
CSS Animations: Create Amazing Effects on Your Website         3676
Using MODX CMS to Build Websites: A Beginner's Guide           3677
Length: 3678, dtype: int64

In [57]:
indices['Anatomy and Figure Drawing for Beginners']

1195

In [58]:
sig[1195]

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.05619391])

In [59]:
list(enumerate(sig[indices['Anatomy and Figure Drawing for Beginners']]))

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.07455857505869426),
 (4, 0.0),
 (5, 0.0),
 (6, 0.05828313350846593),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.050005652999740705),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.06054904757478153),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.058927394348005833),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77

In [65]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the courses
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar courses
    sig_scores = sig_scores[1:11]

    # course indices
    course_indices = [i[0] for i in sig_scores]

    # Top 10 most similar courses
    return finaldata['course_title'].iloc[course_indices]

In [66]:
# Testing our course recommendation system 
give_rec('Anatomy and Figure Drawing for Beginners')

1214    Anatomy for Figure Drawing: Mastering the Huma...
1658                The Complete Figure Drawing Course HD
1286    Figure Drawing From Life Using The Reilly Tech...
1294    Discover the 7 Secrets to Figure Drawing: Draw...
1218                 How to Draw and Sketch for Beginners
1433    Complete Portrait and Figure Drawing (step-by-...
1231                    Drawing Perspective For Beginners
1500                                        Corel Draw x7
1757                         From Drawing to Illustration
2954                                   CSS3 for beginners
Name: course_title, dtype: object

In [67]:
give_rec('Forex Traders - Create Your First Trading Robot. No Coding!')

64      Algorithmic Trading In Forex: Create Your Firs...
117     Forex Trading Secrets |MT4 Robot (EA) inside, ...
1126                            The Complete Forex Trader
1120             Build your own Naked Trading Forex Robot
562     MQL4 Programming for Traders: Build Robust Tra...
347     Become A Forex Trader - Forex trading like a p...
469                                         Forex Trading
47      Forex Robots: Expect To Earn 175% P.A. Forex R...
2998    How to Create a Wordpress Website from Scratch...
3478      Create an E-commerce website without any coding
Name: course_title, dtype: object

In [68]:
give_rec('Projects in Django and Python')

3194              Learn Python Django - A Hands-On Course
2570    Coding for Entrepreneurs: Learn Python, Django...
3332                                 Django For Beginners
2528          Learn Python and Django: Payment Processing
2481                                    Projects in HTML5
3200    Python and Django Full Stack Web Developer Boo...
3326    Core: A Web App Reference Guide for Django, Py...
3392    Python Programming: Create an Digital Marketpl...
3197    Try Django 1.9 | Build a Blog and Learn Python...
2868                                    Django by Example
Name: course_title, dtype: object